# 1. Setup and Imports

In [2]:
!pip install trulens-core  -q
!pip install trulens-providers-cortex  -q
!pip install trulens-connectors-snowflake -q
!pip install llama-index -q
!pip install llama-index-embeddings-huggingface -q
!pip install snowflake-sqlalchemy -q
!pip install snowflake -q
!pip install snowflake-snowpark-python requests ipython -q
!pip install snowflake-snowpark-python snowflake-core -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 82.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dep

In [ ]:
from trulens.connectors.snowflake import SnowflakeConnector
from snowflake.snowpark.context import get_active_session
from trulens.providers.cortex.provider import Cortex
from trulens.apps.custom import TruCustomApp
from trulens.apps.custom import instrument
from snowflake.snowpark import Session
from contextlib import contextmanager
from trulens.core import TruSession
from trulens.core import Feedback
from snowflake.core import Root
from trulens.core import Select
from functools import partial
from getpass import getpass
import snowflake.connector
from typing import List
import pandas as pd
import numpy as np
import json
import time
import os



# 2. Configuration

In [52]:
CORTEX_SEARCH_DATABASE = "Replace with your database"  
CORTEX_SEARCH_SCHEMA = "Replace with your schema"
CORTEX_SEARCH_SERVICE = "Replace with your cortex search service"
COLUMNS = [
    "chunk",
    "relative_path",
    "category"
]

# Snowflake Connection and Session
connection_params = {
    "account": "Replace with your Snowflake account",  
    "user": "Replace with your Snowflake user",  
    "password": "Replace with your Snowflake password",  
    "warehouse": "Replace with your Snowflake warehouse", 
    "database": "Replace with your Snowflake database",  
    "schema": "Replace with your Snowflake schema", 
    "role": "Replace with your Snowflake role",  
}
# NUM_CHUNKS = 1
SLIDE_WINDOW = 10

In [53]:
# Get active Snowflake session
session = Session.builder.configs(connection_params).create()
root = Root(session)
svc = root.databases[CORTEX_SEARCH_DATABASE].schemas[CORTEX_SEARCH_SCHEMA].cortex_search_services[CORTEX_SEARCH_SERVICE]

In [54]:
snowpark_session = Session.builder.configs(connection_params).create()

# 3. experimenting with different numbers of retrieved chunks in my RAG

In [55]:
NUM_CHUNKS = 3
class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = NUM_CHUNKS):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(self._snowpark_session)
        cortex_search_service = (
            root.databases["CC_QUICKSTART_CORTEX_SEARCH_DOCS_1"]
            .schemas["DATA"]
            .cortex_search_services["CC_SEARCH_SERVICE_CS"]  # Fixed service name to match what you created
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["chunk"],  # Retrieve the 'chunk' column instead of 'doc_text'
            limit=self._limit_to_retrieve,
        )

        if resp.results:
            return [curr["chunk"] for curr in resp.results]  # Access the 'chunk' column
        else:
            return []

retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=NUM_CHUNKS)

retrieved_context = retriever.retrieve(query="what is the definition of database?")
retrieved_context

['the DBMS.\\n  \\nData Storage Data Management IDE RDBMS Data Security (Software application) Queries AdministrationView Mechanism allows each user to have his or her own view of the database  \\nSource: https://vertabelo.com/blog/what-is-rdbms/University of Ottawa, ELG 5166, Fall 2023\\n# Data Validation using Schemas\\n\\n* .: Set of rules that govern a database usually written in a data definition language such as SQL, JSON Schema, or XML DTD\\n  \\nLogical schema Entities Attributes Relationships Physical schema\\n* : Includes data validation definition for columns or attributes, data format, relationship between entities.\\n* . Typically defined by the database designer through a process called data modeling.\\n* : In a relational database, schema definitions are often called the Entity-Relationship diagram or schema diagram.\\n* . Schemas are handled differently in a distributed database.\\n\\n# RDMS Schemas Entity-Relationship Diagram\\n  \\nTicket System-Visual Paradigm Enterp

In [ ]:
tru_snowflake_connector = SnowflakeConnector(snowpark_session=snowpark_session)

tru_session = TruSession(connector=tru_snowflake_connector)


In [57]:
NUM_CHUNKS = 3
class RAG_from_scratch:

    def __init__(self, snowpark_session, use_chat_history=True, course_content="ALL"):
        """
        Initialize the RAG system.

        Args:
            snowpark_session: Snowpark session object.
            use_chat_history: Whether to use chat history for context.
            course_content: The course content category to filter results (default: "ALL").
        """
        self.retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=NUM_CHUNKS)
        self.use_chat_history = use_chat_history
        self.course_content = course_content
        self.messages = []  # Chat history

    @instrument
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from the Cortex Search Service based on the query and course content.
        """
        if self.course_content == "ALL":
            response = svc.search(query, COLUMNS, limit=NUM_CHUNKS)
        else:
            filter_obj = {"@eq": {"category": self.course_content}}
            response = svc.search(query, COLUMNS, filter=filter_obj, limit=NUM_CHUNKS)

        if response.results:
            return [{"chunk": result["chunk"], "relative_path": result["relative_path"]} for result in response.results]
        else:
            return []

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate an answer from the context using Snowflake Cortex with Mistral Large 2.
        """
        # Create the prompt with context and chat history
        prompt = self.create_prompt(query, context_str)

        # Use Snowflake Cortex to generate the completion
        cmd = "select snowflake.cortex.complete(?, ?) as response"
        df_response = session.sql(cmd, params=['mistral-large2', prompt]).collect()
        return df_response[0].RESPONSE

    @instrument
    def create_prompt(self, query: str, context_str: list) -> str:
        """
        Create a prompt for the LLM with context from search results and chat history.
        """
        if self.use_chat_history:
            chat_history = self.get_chat_history()
            if chat_history:
                question_summary = self.summarize_question_with_history(chat_history, query)
                context_str = self.retrieve_context(question_summary)
            else:
                context_str = self.retrieve_context(query)
        else:
            context_str = self.retrieve_context(query)
            chat_history = ""

        # Format the context into a string
        context_formatted = "\n".join([f"Chunk: {item['chunk']}\nSource: {item['relative_path']}" for item in context_str])

        # Build the prompt
        prompt = f"""
        I am IKNOW, a friendly and witty lecture assistant specializing in helping with {self.course_content} topics! I love assisting students by explaining concepts and finding the best study resources from our collection.

        **Important Instructions:**
        - Only provide information related to {self.course_content}.
        - If the query is not related to {self.course_content}, respond with: "I'm sorry, I can only assist with topics related to {self.course_content}."
        - Prioritize topics that align with the syllabus or current lecture needs.
        - List all matching topics or resources as numbered options.
        - Ask which topic they'd like to explore in more detail.
        - Ensure the answer is short and comprehensive.
        - Write each point in a separate line.

        <chat_history>
        {chat_history}
        </chat_history>

        <context>
        {context_formatted}
        </context>

        User Query: {query}
        Current Course_Content: {self.course_content}

        Response (as IKNOW, study partner):
        """
        return prompt

    @instrument
    def summarize_question_with_history(self, chat_history: list, question: str) -> str:
        """
        Summarize the chat history and current question for better context.
        """
        prompt = f"""
        Based on the chat history below and the question, generate a query that extends the question
        with the chat history provided. The query should be in natural language.
        Answer with only the query. Do not add any explanation.
        <chat_history>
        {chat_history}
        </chat_history>
        <question>
        {question}
        </question>
        """
        cmd = "select snowflake.cortex.complete(?, ?) as response"
        df_response = session.sql(cmd, params=['mistral-large2', prompt]).collect()
        summary = df_response[0].RESPONSE
        return summary.replace("'", "")

    @instrument
    def get_chat_history(self) -> str:
        """
        Retrieve recent messages from the chat history.
        """
        start_index = max(0, len(self.messages) - SLIDE_WINDOW)
        return "\n".join([msg["content"] for msg in self.messages[start_index:-1]])

    @instrument
    def query(self, query: str) -> str:
        """
        Query the RAG system with a user question.
        """
        # Retrieve context based on the query
        context_str = self.retrieve_context(query)

        # Generate a completion using the context
        response = self.generate_completion(query, context_str)

        # Update chat history
        self.messages.append({"role": "user", "content": query})
        self.messages.append({"role": "assistant", "content": response})

        return response


# Initialize the RAG system
rag = RAG_from_scratch(snowpark_session=snowpark_session, use_chat_history=True, course_content="weektwo")

decorating <function RAG_from_scratch.retrieve_context at 0x7c4c7017f400>
decorating <function RAG_from_scratch.generate_completion at 0x7c4c7017f1c0>
decorating <function RAG_from_scratch.create_prompt at 0x7c4c7017e830>
decorating <function RAG_from_scratch.summarize_question_with_history at 0x7c4c703bee60>
decorating <function RAG_from_scratch.get_chat_history at 0x7c4c7017c310>
decorating <function RAG_from_scratch.query at 0x7c4c7017e440>
adding method <class '__main__.RAG_from_scratch'> retrieve_context __main__
adding method <class '__main__.RAG_from_scratch'> generate_completion __main__
adding method <class '__main__.RAG_from_scratch'> create_prompt __main__
adding method <class '__main__.RAG_from_scratch'> summarize_question_with_history __main__
adding method <class '__main__.RAG_from_scratch'> get_chat_history __main__
adding method <class '__main__.RAG_from_scratch'> query __main__


In [58]:
NUM_CHUNKS=5
class RAG_from_scratch_two:

    def __init__(self, snowpark_session, use_chat_history=True, course_content="ALL"):
        """
        Initialize the RAG system.

        Args:
            snowpark_session: Snowpark session object.
            use_chat_history: Whether to use chat history for context.
            course_content: The course content category to filter results (default: "ALL").
        """
        self.retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=NUM_CHUNKS)
        self.use_chat_history = use_chat_history
        self.course_content = course_content
        self.messages = []  # Chat history

    @instrument
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from the Cortex Search Service based on the query and course content.
        """
        if self.course_content == "ALL":
            response = svc.search(query, COLUMNS, limit=NUM_CHUNKS)
        else:
            filter_obj = {"@eq": {"category": self.course_content}}
            response = svc.search(query, COLUMNS, filter=filter_obj, limit=NUM_CHUNKS)

        if response.results:
            return [{"chunk": result["chunk"], "relative_path": result["relative_path"]} for result in response.results]
        else:
            return []

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate an answer from the context using Snowflake Cortex with Mistral Large 2.
        """
        # Create the prompt with context and chat history
        prompt = self.create_prompt(query, context_str)

        # Use Snowflake Cortex to generate the completion
        cmd = "select snowflake.cortex.complete(?, ?) as response"
        df_response = session.sql(cmd, params=['mistral-large2', prompt]).collect()
        return df_response[0].RESPONSE

    @instrument
    def create_prompt(self, query: str, context_str: list) -> str:
        """
        Create a prompt for the LLM with context from search results and chat history.
        """
        if self.use_chat_history:
            chat_history = self.get_chat_history()
            if chat_history:
                question_summary = self.summarize_question_with_history(chat_history, query)
                context_str = self.retrieve_context(question_summary)
            else:
                context_str = self.retrieve_context(query)
        else:
            context_str = self.retrieve_context(query)
            chat_history = ""

        # Format the context into a string
        context_formatted = "\n".join([f"Chunk: {item['chunk']}\nSource: {item['relative_path']}" for item in context_str])

        # Build the prompt
        prompt = f"""
        I am IKNOW, a friendly and witty lecture assistant specializing in helping with {self.course_content} topics! I love assisting students by explaining concepts and finding the best study resources from our collection.

        **Important Instructions:**
        - Only provide information related to {self.course_content}.
        - If the query is not related to {self.course_content}, respond with: "I'm sorry, I can only assist with topics related to {self.course_content}."
        - Prioritize topics that align with the syllabus or current lecture needs.
        - List all matching topics or resources as numbered options.
        - Ask which topic they'd like to explore in more detail.
        - Ensure the answer is short and comprehensive.
        - Write each point in a separate line.

        <chat_history>
        {chat_history}
        </chat_history>

        <context>
        {context_formatted}
        </context>

        User Query: {query}
        Current Course_Content: {self.course_content}

        Response (as IKNOW, study partner):
        """
        return prompt

    @instrument
    def summarize_question_with_history(self, chat_history: list, question: str) -> str:
        """
        Summarize the chat history and current question for better context.
        """
        prompt = f"""
        Based on the chat history below and the question, generate a query that extends the question
        with the chat history provided. The query should be in natural language.
        Answer with only the query. Do not add any explanation.
        <chat_history>
        {chat_history}
        </chat_history>
        <question>
        {question}
        </question>
        """
        cmd = "select snowflake.cortex.complete(?, ?) as response"
        df_response = session.sql(cmd, params=['mistral-large2', prompt]).collect()
        summary = df_response[0].RESPONSE
        return summary.replace("'", "")

    @instrument
    def get_chat_history(self) -> str:
        """
        Retrieve recent messages from the chat history.
        """
        start_index = max(0, len(self.messages) - SLIDE_WINDOW)
        return "\n".join([msg["content"] for msg in self.messages[start_index:-1]])

    @instrument
    def query(self, query: str) -> str:
        """
        Query the RAG system with a user question.
        """
        # Retrieve context based on the query
        context_str = self.retrieve_context(query)

        # Generate a completion using the context
        response = self.generate_completion(query, context_str)

        # Update chat history
        self.messages.append({"role": "user", "content": query})
        self.messages.append({"role": "assistant", "content": response})

        return response


# Initialize the RAG system
rag_two = RAG_from_scratch_two(snowpark_session=snowpark_session, use_chat_history=True, course_content="weektwo")

decorating <function RAG_from_scratch_two.retrieve_context at 0x7c4c7017fac0>
decorating <function RAG_from_scratch_two.generate_completion at 0x7c4c7017c790>
decorating <function RAG_from_scratch_two.create_prompt at 0x7c4c7017c0d0>
decorating <function RAG_from_scratch_two.summarize_question_with_history at 0x7c4c7017e4d0>
decorating <function RAG_from_scratch_two.get_chat_history at 0x7c4c7017e9e0>
decorating <function RAG_from_scratch_two.query at 0x7c4c7017f520>
adding method <class '__main__.RAG_from_scratch_two'> retrieve_context __main__
adding method <class '__main__.RAG_from_scratch_two'> generate_completion __main__
adding method <class '__main__.RAG_from_scratch_two'> create_prompt __main__
adding method <class '__main__.RAG_from_scratch_two'> summarize_question_with_history __main__
adding method <class '__main__.RAG_from_scratch_two'> get_chat_history __main__
adding method <class '__main__.RAG_from_scratch_two'> query __main__


In [59]:
NUM_CHUNKS=7
class RAG_from_scratch_three:

    def __init__(self, snowpark_session, use_chat_history=True, course_content="ALL"):
        """
        Initialize the RAG system.

        Args:
            snowpark_session: Snowpark session object.
            use_chat_history: Whether to use chat history for context.
            course_content: The course content category to filter results (default: "ALL").
        """
        self.retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=NUM_CHUNKS)
        self.use_chat_history = use_chat_history
        self.course_content = course_content
        self.messages = []  # Chat history

    @instrument
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from the Cortex Search Service based on the query and course content.
        """
        if self.course_content == "ALL":
            response = svc.search(query, COLUMNS, limit=NUM_CHUNKS)
        else:
            filter_obj = {"@eq": {"category": self.course_content}}
            response = svc.search(query, COLUMNS, filter=filter_obj, limit=NUM_CHUNKS)

        if response.results:
            return [{"chunk": result["chunk"], "relative_path": result["relative_path"]} for result in response.results]
        else:
            return []

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate an answer from the context using Snowflake Cortex with Mistral Large 2.
        """
        # Create the prompt with context and chat history
        prompt = self.create_prompt(query, context_str)

        # Use Snowflake Cortex to generate the completion
        cmd = "select snowflake.cortex.complete(?, ?) as response"
        df_response = session.sql(cmd, params=['mistral-large2', prompt]).collect()
        return df_response[0].RESPONSE

    @instrument
    def create_prompt(self, query: str, context_str: list) -> str:
        """
        Create a prompt for the LLM with context from search results and chat history.
        """
        if self.use_chat_history:
            chat_history = self.get_chat_history()
            if chat_history:
                question_summary = self.summarize_question_with_history(chat_history, query)
                context_str = self.retrieve_context(question_summary)
            else:
                context_str = self.retrieve_context(query)
        else:
            context_str = self.retrieve_context(query)
            chat_history = ""

        # Format the context into a string
        context_formatted = "\n".join([f"Chunk: {item['chunk']}\nSource: {item['relative_path']}" for item in context_str])

        # Build the prompt
        prompt = f"""
        I am IKNOW, a friendly and witty lecture assistant specializing in helping with {self.course_content} topics! I love assisting students by explaining concepts and finding the best study resources from our collection.

        **Important Instructions:**
        - Only provide information related to {self.course_content}.
        - If the query is not related to {self.course_content}, respond with: "I'm sorry, I can only assist with topics related to {self.course_content}."
        - Prioritize topics that align with the syllabus or current lecture needs.
        - List all matching topics or resources as numbered options.
        - Ask which topic they'd like to explore in more detail.
        - Ensure the answer is short and comprehensive.
        - Write each point in a separate line.

        <chat_history>
        {chat_history}
        </chat_history>

        <context>
        {context_formatted}
        </context>

        User Query: {query}
        Current Course_Content: {self.course_content}

        Response (as IKNOW, study partner):
        """
        return prompt

    @instrument
    def summarize_question_with_history(self, chat_history: list, question: str) -> str:
        """
        Summarize the chat history and current question for better context.
        """
        prompt = f"""
        Based on the chat history below and the question, generate a query that extends the question
        with the chat history provided. The query should be in natural language.
        Answer with only the query. Do not add any explanation.
        <chat_history>
        {chat_history}
        </chat_history>
        <question>
        {question}
        </question>
        """
        cmd = "select snowflake.cortex.complete(?, ?) as response"
        df_response = session.sql(cmd, params=['mistral-large2', prompt]).collect()
        summary = df_response[0].RESPONSE
        return summary.replace("'", "")

    @instrument
    def get_chat_history(self) -> str:
        """
        Retrieve recent messages from the chat history.
        """
        start_index = max(0, len(self.messages) - SLIDE_WINDOW)
        return "\n".join([msg["content"] for msg in self.messages[start_index:-1]])

    @instrument
    def query(self, query: str) -> str:
        """
        Query the RAG system with a user question.
        """
        # Retrieve context based on the query
        context_str = self.retrieve_context(query)

        # Generate a completion using the context
        response = self.generate_completion(query, context_str)

        # Update chat history
        self.messages.append({"role": "user", "content": query})
        self.messages.append({"role": "assistant", "content": response})

        return response


# Initialize the RAG system
rag_three = RAG_from_scratch_two(snowpark_session=snowpark_session, use_chat_history=True, course_content="weektwo")

decorating <function RAG_from_scratch_three.retrieve_context at 0x7c4c7017f5b0>
decorating <function RAG_from_scratch_three.generate_completion at 0x7c4c7017e8c0>
decorating <function RAG_from_scratch_three.create_prompt at 0x7c4c70249a20>
decorating <function RAG_from_scratch_three.summarize_question_with_history at 0x7c4c7017e560>
decorating <function RAG_from_scratch_three.get_chat_history at 0x7c4c7017d7e0>
decorating <function RAG_from_scratch_three.query at 0x7c4c5bf581f0>
adding method <class '__main__.RAG_from_scratch_three'> retrieve_context __main__
adding method <class '__main__.RAG_from_scratch_three'> generate_completion __main__
adding method <class '__main__.RAG_from_scratch_three'> create_prompt __main__
adding method <class '__main__.RAG_from_scratch_three'> summarize_question_with_history __main__
adding method <class '__main__.RAG_from_scratch_three'> get_chat_history __main__
adding method <class '__main__.RAG_from_scratch_three'> query __main__


# 4. Evaluation

In [60]:
provider = Cortex(snowpark_session, "mistral-large2")

In [61]:
def groundedness_feedback():
    return Feedback(
        provider.groundedness_measure_with_cot_reasons,
        name="Groundedness"
    )

f_groundedness = groundedness_feedback()



f_groundedness = (
    Feedback(partial(provider.groundedness_measure_with_cot_reasons), name="Groundedness")
    .on(Select.RecordCalls.retrieve_context.rets[:].collect())
    .on_output()
)

f_answer_relevance = (
    Feedback(provider.relevance, name="Answer Relevance")
    .on_input()
    .on_output()
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve_context.rets[:].collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [62]:
prompts = [
    "what is the meaning of  Database Management System?",
    "what is the meaning of database?",
    "what is the meaning of  Application Program?"
]

In [63]:
# Initialize the TruLens app with feedback functions
tru_rag = TruCustomApp(
    rag,  # Your RAG instance
    app_name="RAGV1",  # Name of the app
    app_version="V1",  # Version of the app
    feedbacks=[f_groundedness, f_answer_relevance]  # List of feedback functions
)

instrumenting <class '__main__.RAG_from_scratch'> for base <class '__main__.RAG_from_scratch'>
	instrumenting retrieve_context
	instrumenting generate_completion
	instrumenting create_prompt
	instrumenting summarize_question_with_history
	instrumenting get_chat_history
	instrumenting query
skipping base <class 'object'> because of class
skipping base <class '__main__.CortexSearchRetriever'> because of class
skipping base <class 'object'> because of class


In [6]:
# Run in smaller batches
batch_size = 5
for i in range(0, len(prompts), batch_size):
    batch = prompts[i:i+batch_size]
    
    try:
        with tru_rag as recording:
            for prompt in batch:
                rag.query(prompt)
                
        # Add delay between batches
        time.sleep(1) 
        
    except Exception as e:
        print(f"Error processing batch {i//batch_size}: {str(e)}")
        continue

# Try getting leaderboard separately
try:
    leaderboard = tru_session.get_leaderboard()
    print(leaderboard)
except Exception as e:
    print(f"Error getting leaderboard: {str(e)}")


calling <function RAG_from_scratch.query at 0x7c4c7017e440> with (<__main__.RAG_from_scratch object at 0x7c4c7025d120>, 'what is the meaning of  Database Management System?')
calling <function RAG_from_scratch.retrieve_context at 0x7c4c7017f400> with (<__main__.RAG_from_scratch object at 0x7c4c7025d120>, 'what is the meaning of  Database Management System?')
calling <function RAG_from_scratch.generate_completion at 0x7c4c7017f1c0> with (<__main__.RAG_from_scratch object at 0x7c4c7025d120>, 'what is the meaning of  Database Management System?', [])
calling <function RAG_from_scratch.create_prompt at 0x7c4c7017e830> with (<__main__.RAG_from_scratch object at 0x7c4c7025d120>, 'what is the meaning of  Database Management System?', [])
calling <function RAG_from_scratch.get_chat_history at 0x7c4c7017c310> with (<__main__.RAG_from_scratch object at 0x7c4c7025d120>,)
calling <function RAG_from_scratch.retrieve_context at 0x7c4c7017f400> with (<__main__.RAG_from_scratch object at 0x7c4c7025d1

In [65]:
tru_rag = TruCustomApp(
    rag_two,
    app_name="RAGV2",
    app_version="V2",
    feedbacks=[f_groundedness, f_answer_relevance]
    )


instrumenting <class '__main__.RAG_from_scratch_two'> for base <class '__main__.RAG_from_scratch_two'>
	instrumenting retrieve_context
	instrumenting generate_completion
	instrumenting create_prompt
	instrumenting summarize_question_with_history
	instrumenting get_chat_history
	instrumenting query
skipping base <class 'object'> because of class
skipping base <class '__main__.CortexSearchRetriever'> because of class
skipping base <class 'object'> because of class


In [5]:
# Run in smaller batches
batch_size = 5
for i in range(0, len(prompts), batch_size):
    batch = prompts[i:i+batch_size]
    
    try:
        with tru_rag as recording:
            for prompt in batch:
                rag_two.query(prompt)
                
        # Add delay between batches
        time.sleep(1) 
        
    except Exception as e:
        print(f"Error processing batch {i//batch_size}: {str(e)}")
        continue

# Try getting leaderboard separately
try:
    leaderboard = tru_session.get_leaderboard()
    print(leaderboard)
except Exception as e:
    print(f"Error getting leaderboard: {str(e)}")


calling <function RAG_from_scratch_two.query at 0x7c4c7017f520> with (<__main__.RAG_from_scratch_two object at 0x7c4c70157af0>, 'what is the meaning of  Database Management System?')
calling <function RAG_from_scratch_two.retrieve_context at 0x7c4c7017fac0> with (<__main__.RAG_from_scratch_two object at 0x7c4c70157af0>, 'what is the meaning of  Database Management System?')
calling <function RAG_from_scratch_two.generate_completion at 0x7c4c7017c790> with (<__main__.RAG_from_scratch_two object at 0x7c4c70157af0>, 'what is the meaning of  Database Management System?', [])
calling <function RAG_from_scratch_two.create_prompt at 0x7c4c7017c0d0> with (<__main__.RAG_from_scratch_two object at 0x7c4c70157af0>, 'what is the meaning of  Database Management System?', [])
calling <function RAG_from_scratch_two.get_chat_history at 0x7c4c7017e9e0> with (<__main__.RAG_from_scratch_two object at 0x7c4c70157af0>,)
calling <function RAG_from_scratch_two.retrieve_context at 0x7c4c7017fac0> with (<__ma

In [12]:
tru_rag = TruCustomApp(
    rag_three,
    app_name="RAGV3",
    app_version="V3",
    feedbacks=[f_groundedness, f_answer_relevance]
    )



instrumenting <class '__main__.RAG_from_scratch_three'> for base <class '__main__.RAG_from_scratch_three'>
	instrumenting retrieve_context
	instrumenting generate_completion
	instrumenting create_prompt
	instrumenting summarize_question_with_history
	instrumenting get_chat_history
	instrumenting query
skipping base <class 'object'> because of class
skipping base <class '__main__.CortexSearchRetriever'> because of class
skipping base <class 'object'> because of class




In [10]:
# Run in smaller batches
batch_size = 5
for i in range(0, len(prompts), batch_size):
    batch = prompts[i:i+batch_size]
    
    try:
        with tru_rag as recording:
            for prompt in batch:
                rag_three.query(prompt)
                
        # Add delay between batches
        time.sleep(1) 
        
    except Exception as e:
        print(f"Error processing batch {i//batch_size}: {str(e)}")
        continue

# Try getting leaderboard separately
try:
    leaderboard = tru_session.get_leaderboard()
    print(leaderboard)
except Exception as e:
    print(f"Error getting leaderboard: {str(e)}")


calling <function RAG_from_scratch_three.query at 0x7c4c7017f520> with (<__main__.RAG_from_scratch_three object at 0x7c4c7025ef50>, 'what is the meaning of  Database Management System?')
calling <function RAG_from_scratch_three.retrieve_context at 0x7c4c7017fac0> with (<__main__.RAG_from_scratch_three object at 0x7c4c7025ef50>, 'what is the meaning of  Database Management System?')
calling <function RAG_from_scratch_three.generate_completion at 0x7c4c7017c790> with (<__main__.RAG_from_scratch_three object at 0x7c4c7025ef50>, 'what is the meaning of  Database Management System?', [])
calling <function RAG_from_scratch_three.create_prompt at 0x7c4c7017c0d0> with (<__main__.RAG_from_scratch_three object at 0x7c4c7025ef50>, 'what is the meaning of  Database Management System?', [])
calling <function RAG_from_scratch_three.get_chat_history at 0x7c4c7017e9e0> with (<__main__.RAG_from_scratch_three object at 0x7c4c7025ef50>,)
calling <function RAG_from_scratch_three.summarize_question_with_h